In [54]:
import numpy as np
import pygame as pg
from pygame.locals import *

In [55]:
Nx = 8
grid_size = 64
Lx = 600
Ly = 600
side = 1
max_eva = 100
red_win = False
white_win = False
alpha = -200.
beta = -alpha

margin_x = (Lx-Nx*grid_size)//2
margin_y = (Ly-Nx*grid_size)//2

step_fr_w = 9 #forward_right
step_fl_w = 7 #forward_left
step_br_w = -7 #backward_rigt
step_bl_w = -9 #backward_left

step_fr_r = -7 #forward_right
step_fl_r = -9 #forward_left
step_br_r = 9 #backward_right
step_bl_r = 7 #backward_left

### Initialize the board

In [56]:
iE_w = np.arange(0,12)
iE_r = np.arange(20,32)

x_w = (2*iE_w)%Nx
y_w = ((2*iE_w)//Nx)
x_r = (2*iE_r)%Nx
y_r = ((2*iE_r)//Nx)

parity = (x_w+y_w+1)%2
x_w += parity
parity = (x_r+y_r+1)%2
x_r += parity

man_w = x_w+ Nx*y_w
man_w = list(man_w)
man_r = x_r+ Nx*y_r
man_r = list(man_r)

king_w = []
king_r = []

print text [Reference](https://home.gamer.com.tw/creationDetail.php?sn=3669589)

In [75]:
def CHECKER_LIST(list_of_checkers):
    for element in man_r:
        list_of_checkers[element] = 'm_r'
    for element in man_w:
        list_of_checkers[element] = 'm_w'
    for element in king_r:
        list_of_checkers[element] = 'k_r'
    for element in king_w:
        list_of_checkers[element] = 'k_w'
    return(list_of_checkers)
def CHECKER_LIST1(list_of_checkers1):
    for element in man_r:
        list_of_checkers1[element] = 'm_r'
    for element in man_w:
        list_of_checkers1[element] = 'm_w'
    for element in king_r:
        list_of_checkers1[element] = 'k_r'
    for element in king_w:
        list_of_checkers1[element] = 'k_w'
    return(list_of_checkers1)

def LIST_CHECKER(list_of_checkers, man_r, man_w, king_r, king_w):
    for index, element in enumerate(list_of_checkers):
        if element == 'm_r':
            man_r.append(index)
        elif element == 'm_w':
            man_w.append(index)
        elif element == 'k_r':
            king_r.append(index)
        elif element == 'k_w':
            king_w.append(index)
    return(man_r, man_w, king_r, king_w)

In [58]:
def MAN_JUMP_RED(index, list_of_checkers, man_jump, taken_by_man):
    if index%Nx != Nx-1:
        temp = index + step_fr_r
        temp2 = temp + step_fr_r
        if index//Nx>1 and temp%Nx!=Nx-1 and 'w' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_man.append(temp)
            man_jump.append( (index,temp2) )
    if index%Nx != 0:
        temp = index + step_fl_r
        temp2 = temp + step_fl_r
        if index//Nx>1 and temp%Nx!=0 and 'w' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_man.append(temp)
            man_jump.append( (index,temp2) )
            
def MAN_WALK_RED(index, list_of_checkers, man_walk):
    if index%Nx != Nx-1:
        temp = index + step_fr_r
        if list_of_checkers[temp] == '':
            man_walk.append( (index,temp) )
    if index%Nx != 0:
        temp = index + step_fl_r
        if list_of_checkers[temp] == '':
            man_walk.append( (index,temp) )

def KING_JUMP_RED(index, list_of_checkers, king_jump, taken_by_king):
    if index%Nx != Nx-1:
        temp = index + step_fr_r
        temp2 = temp + step_fr_r
        if index//Nx>1 and temp%Nx!=Nx-1 and 'w' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_king.append(temp)
            king_jump.append( (index,temp2) )                  
        temp = index + step_br_r
        temp2 = temp + step_br_r
        if index//Nx<Nx-2 and temp%Nx!=Nx-1 and 'w' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_king.append(temp)
            king_jump.append( (index,temp2) )
    if index%Nx != 0:
        temp = index + step_fl_r
        temp2 = temp + step_fl_r
        if index//Nx>1 and temp%Nx!=0 and 'w' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_king.append(temp)
            king_jump.append( (index,temp2) )
        temp = index + step_bl_r
        temp2 = temp + step_bl_r
        if index//Nx<Nx-2 and temp%Nx!=0 and 'w' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_king.append(temp)
            king_jump.append( (index,temp2) )

def KING_WALK_RED(index, list_of_checkers, king_walk):
    if index%Nx != Nx-1:
        temp = index + step_fr_r
        if index//Nx>0 and list_of_checkers[temp] == '':
            king_walk.append( (index,temp) )
        temp = index + step_br_r
        if index//Nx<Nx-1 and list_of_checkers[temp] == '':
            king_walk.append( (index,temp) )
    if index%Nx != 0:
        temp = index + step_fl_r
        if index//Nx>0 and list_of_checkers[temp] == '':
            king_walk.append( (index,temp) )
        temp = index + step_bl_r
        if index//Nx<Nx-1 and list_of_checkers[temp] == '':
            king_walk.append( (index,temp) )

In [59]:
def MAN_JUMP_WHITE(index, list_of_checkers, man_jump, taken_by_man):
    if index%Nx != Nx-1:
        temp = index + step_fr_w
        temp2 = temp + step_fr_w
        if index//Nx<Nx-2 and temp%Nx!=Nx-1 and 'r' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_man.append(temp)
            man_jump.append( (index,temp2) )
    if index%Nx != 0:
        temp = index + step_fl_w
        temp2 = temp + step_fl_w
        if index//Nx<Nx-2 and temp%Nx!=0 and 'r' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_man.append(temp)
            man_jump.append( (index,temp2) )

def MAN_WALK_WHITE(index, list_of_checkers, man_walk):
    if index%Nx != Nx-1:
        temp = index + step_fr_w
        if list_of_checkers[temp] == '':
            man_walk.append( (index,temp) )
    if index%Nx != 0:
        temp = index + step_fl_w
        if list_of_checkers[temp] == '':
            man_walk.append( (index,temp) )
                    
def KING_JUMP_WHITE(index, list_of_checkers, king_jump, taken_by_king):
    if index%Nx != Nx-1:
        temp = index + step_fr_w
        temp2 = temp + step_fr_w
        if index//Nx<Nx-2 and temp%Nx!=Nx-1 and 'r' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_king.append(temp)
            king_jump.append( (index,temp2) )                  
        temp = index + step_br_w
        temp2 = temp + step_br_w
        if index//Nx>1 and temp%Nx!=Nx-1 and 'r' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_king.append(temp)
            king_jump.append( (index,temp2) )                 
    if index%Nx != 0:
        temp = index + step_fl_w
        temp2 = temp + step_fl_w
        if index//Nx<Nx-2 and temp%Nx!=0 and 'r' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_king.append(temp)
            king_jump.append( (index,temp2) )
        temp = index + step_bl_w
        temp2 = temp + step_bl_w
        if index//Nx>1 and temp%Nx!=0 and 'r' in list_of_checkers[temp] and list_of_checkers[temp2] == '':
            #record the position of taken pieces
            taken_by_king.append(temp)
            king_jump.append( (index,temp2) )

def KING_WALK_WHITE(index, list_of_checkers, king_walk):
    if index%Nx != Nx-1:
        temp = index + step_fr_w
        if index//Nx<Nx-1 and list_of_checkers[temp] == '':
            king_walk.append( (index,temp) )
        temp = index + step_br_w
        if index//Nx>0 and list_of_checkers[temp] == '':
            king_walk.append( (index,temp) )
    if index%Nx != 0:
        temp = index + step_fl_w
        if index//Nx<Nx-1 and list_of_checkers[temp] == '':
            king_walk.append( (index,temp) )
        temp = index + step_bl_w
        if index//Nx>0 and list_of_checkers[temp] == '':
            king_walk.append( (index,temp) )

In [60]:
def AVAILABLE_MOVE(side, list_of_checkers):
    man_jump = []
    king_jump = []
    man_walk = []
    king_walk = []
    taken_by_man = []
    taken_by_king = []
    have_man = []
    have_king = []
#    promote_by_jump = []
#    promote_by_walk = []
    
    # red
    if side==1:
        for index, element in enumerate(list_of_checkers):
            # red man jump
            if element=='m_r':
                MAN_JUMP_RED(index, list_of_checkers, man_jump, taken_by_man)
                have_man.append(index)  #save the position having man so no need to go through the empty sites again 
            # red king jump
            elif element=='k_r':
                KING_JUMP_RED(index, list_of_checkers, king_jump, taken_by_king)
                have_king.append(index)  #save the position having king so no need to go through the empty sites again
        if man_jump == [] and king_jump == []:
            for index in have_man:
                MAN_WALK_RED(index, list_of_checkers, man_walk)
            for index in have_king:
                KING_WALK_RED(index, list_of_checkers, king_walk)
    # white            
    if side==-1:
        for index, element in enumerate(list_of_checkers):
            # white man jump
            if element=='m_w':                 
                MAN_JUMP_WHITE(index, list_of_checkers, man_jump, taken_by_man)
                have_man.append(index)  #save the position having man so no need to go through the empty sites again 
            # white king jump
            elif element=='k_w':
                KING_JUMP_WHITE(index, list_of_checkers, king_jump, taken_by_king)
                have_king.append(index)  #save the position having king so no need to go through the empty sites again
        if man_jump == [] and king_jump == []:
            for index in have_man:
                MAN_WALK_WHITE(index, list_of_checkers, man_walk)
            for index in have_king:
                KING_WALK_WHITE(index, list_of_checkers, king_walk)
                    
    return (man_jump, king_jump, man_walk, king_walk, taken_by_man, taken_by_king)

[Reference 1](https://ithelp.ithome.com.tw/articles/10209416)
[Reference 2](https://gist.github.com/kingknight0921/594251488548e1cdae7c)
[Reference 3](https://ithelp.ithome.com.tw/articles/10209416)

In [61]:
def SHOW_TEXT(font, text, x, y):
    x = x
    y = y
    text = font.render(text, True, (0, 0, 0))
    WINDOWS.blit(text, (x+margin_x+2, y+margin_y))
    pg.display.update()

def WINDOWS_PLOT(WINDOWS, FLIP):
    BOARD = pg.image.load('./tablero_original_revised.png')
    BOARD = pg.transform.smoothscale(BOARD, (grid_size*Nx, grid_size*Nx))

    BOARD_IMAGE = BOARD.get_rect()
    BOARD_IMAGE.topleft = (margin_x, margin_y)

    #ACKGROUND = pg.Surface(WINDOWS.get_size())
    BACKGROUND = pg.image.load('./background_2.jpg')
    BACKGROUND = pg.transform.smoothscale(BACKGROUND, (Lx,Ly))
    BACKGROUND = BACKGROUND.convert()

    WINDOWS.blit(BACKGROUND, (0,0))
    WINDOWS.blit(BOARD, BOARD_IMAGE)

    font = pg.font.Font("./l_10646.ttf", 10)
    for i in range(0,64):
        if FLIP:
            text = f"{Nx**2-1-i}"
        else:
            text = f"{i}"
        text_x = i%Nx
        text_y = i//Nx
        SHOW_TEXT(font, text, text_x*grid_size, text_y*grid_size)

def CHECKER_PLOT(checkers, WINDOWS, FLIP):
    shift_x = 12
    shift_y = 12

    for (index,element) in enumerate(checkers):
        if (element == 'm_r'):
            CHECKER = pg.image.load('./CHECKER_RED.png')
        elif (element == 'm_w'):
            CHECKER = pg.image.load('./CHECKER_WHITE.png')
        elif (element == 'k_r'):
            CHECKER = pg.image.load('./CHECKER_RED_KING.png')
        elif (element == 'k_w'):
            CHECKER = pg.image.load('./CHECKER_WHITE_KING.png')
        
        checker_x = index%8
        checker_y = index//8
        if FLIP:
            checker_x = Nx-1-checker_x
            checker_y = Nx-1-checker_y
        if (element!=''):
            CHECKER = pg.transform.smoothscale(CHECKER, (2*grid_size, 2*grid_size))
            WINDOWS.blit(CHECKER, (checker_x*grid_size+shift_x,checker_y*grid_size+shift_y))
            
def MOVE_HINT(available_move):
    available_start = []
    available_stop = []
    for i in available_move:
        available_start.append(i[0])
        available_start = list(set(available_start))  # remove the repeating elements
        available_stop.append(i[1])
        available_stop = list(set(available_stop))  # remove the repeating elements
    return (available_start, available_stop)
    
def saving(list_of_checkers, save):
    save.append(list_of_checkers.copy())  # Use copy to clone the element of list. If use '=' directly, what is really cloned
                                          # is the address of the list! See http://www.runoob.com/python3/python3-att-list-copy.html

### Computer use random move

In [10]:
pg.init()

FPS = 10 # frame per second
clock = pg.time.Clock()

WINDOWS = pg.display.set_mode((Lx,Ly))
pg.display.set_caption('English Draughts')

#################
checkers = []
for i in range(0,Nx*Nx):
    checkers.append('')
    
#man_r = [24,10,42,44]
#man_w = [12,1]
#king_r = [26,28]
#king_w = [37]
#################
checkers = CHECKER_LIST(checkers)  # checkers is original a empty list. The function CHECKER_LIST fills 
                                   # the list with the poistion of red and white pieces delcared (by declaring man_r, man_w, king_r, king_w).
WINDOWS_PLOT(WINDOWS, FLIP)
CHECKER_PLOT(checkers, WINDOWS, FLIP)
pg.display.flip()

click= []
save = []
side = 1
running = True
continue_to_jump = False
FLIP = True

if side==1:
    print('Red Turn')
else:
    print('White Trun')
print('==============================================')
saving(checkers, save)
while running:
    event = pg.event.wait()
    mods = pg.key.get_mods()
    if event.type == MOUSEBUTTONDOWN and event.button==1:
        (x,y) = pg.mouse.get_pos()
        column = (x-margin_x)//grid_size
        row = (y-margin_y)//grid_size
        index = column + row*Nx
        if FLIP:
            index = Nx**2-1-index
        click.append(index)
    # take back
    elif not continue_to_jump and event.type == KEYDOWN and mods&pg.KMOD_CTRL:  # See https://www.pygame.org/docs/ref/key.html
        if event.key == pg.K_z:
            if len(save)>1:
                checkers = save[-2].copy()  # Use copy to clone the element of list. If use '=' directly, what is really cloned
                                            # is the address of the list! See http://www.runoob.com/python3/python3-att-list-copy.html
                del save[-1]  # remove the last element
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP)
                pg.display.flip() 
                print('\tRed takes back.\n==============================================\nRed Turn')
#                else:
#                    print('White takes back.\nWhite Turn')
                
    # close the window
    elif event.type == pg.QUIT:
        running = False
        
    if not continue_to_jump:
        (man_jump, king_jump, man_walk, king_walk, taken_by_man, taken_by_king) = AVAILABLE_MOVE(side, checkers)
    available_move = man_jump+king_jump+man_walk+king_walk
    # draw out the hints
    (available_start,available_stop) = MOVE_HINT(available_move)  # find the avaiable start points and stop points, then drawing it out as hint
    if side==1:
        for i in available_start:
            hint_x = i%Nx
            hint_y = i//Nx
            if FLIP:
                hint_x = Nx-1-hint_x
                hint_y = Nx-1-hint_y
            pg.draw.circle(WINDOWS, (0,0,255), 
                           (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
        for i in available_stop:
            hint_x = i%Nx
            hint_y = i//Nx
            if FLIP:
                hint_x = Nx-1-hint_x
                hint_y = Nx-1-hint_y
            pg.draw.circle(WINDOWS, (0,255,0),
                           (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
        pg.display.flip()
    # have a winner
    if available_move == []:
        if side==1:
            white_win = True
            print('No move.')
            print('White win!')
        else:
            red_win = True
            print('No move.')
            print('Red win!')
        break
    # Computer moves
    if side==-1:
        jump = man_jump + king_jump
        walk = man_walk + king_walk
        if jump!= []:
            selected = np.random.randint(len(jump))
            if selected < len(man_jump):
                start = man_jump[selected][0]
                stop = man_jump[selected][1]
                taken = (start+stop)//2
                checkers[start] = ''
                checkers[taken] = ''
                checkers[stop] = 'm_w'               
                man_jump = []
                #king_jump = []
                taken_by_man = []
                #taken_by_king = []
                MAN_JUMP_WHITE(stop, checkers, man_jump, taken_by_man)
                print('\tMan jumps from %d to %d.' %(start,stop))
                if man_jump != []:  # white man has a jump sequence
                    continue_to_jump = True
                    print('\tMan continues to jump.')
                else:  # zero or jump sequence ends
                    if stop//Nx == Nx-1:
                        checkers[stop] = 'k_w' #promote
                        print('\tMan promotes.')
                    continue_to_jump = False
                    print('Red Turn')
                    print('==============================================')
                    saving(checkers,save)
                    click= []
                    side = -side
            else:
                selected -= len(man_jump)
                start = king_jump[selected][0]
                stop = king_jump[selected][1]
                taken = (start+stop)//2
                checkers[start] = ''
                checkers[taken] = ''
                checkers[stop] = 'k_w'
                #man_jump = []
                king_jump = []
                #taken_by_man = []
                taken_by_king = []
                KING_JUMP_WHITE(stop, checkers, king_jump, taken_by_king)
                print('\tKing jumps from %d to %d.' %(start,stop))
                if king_jump != []:
                    continue_to_jump = True
                    print('\tKing continues to jump.')
                else:
                    continue_to_jump = False
                    print('Red Turn')
                    print('==============================================')
                    saving(checkers, save)
                    click= []
                side = -side        
            WINDOWS_PLOT(WINDOWS, FLIP)
            CHECKER_PLOT(checkers, WINDOWS, FLIP) 
            pg.display.flip()
        elif walk!=[]:
            selected = np.random.randint(len(walk))
            if selected < len(man_walk):
                start = man_walk[selected][0]
                stop = man_walk[selected][1]
                checkers[start] = ''
                if stop//Nx == Nx-1:
                    checkers[stop] = 'k_w' #promote
                    print('\tMan walks from %d to %d .' %(start,stop))
                    print('\tMan promotes')
                else:
                    checkers[stop] = 'm_w'
                    print('\tMan walks from %d to %d.' %(start,stop))
                print('Red Trun')
                print('==============================================')
            else:
                selected -= len(man_walk)
                start = king_walk[selected][0]
                stop = king_walk[selected][1]
                checkers[start] = ''
                checkers[stop] = 'k_w'
                print('\tKing walks from %d to %d.' %(start,stop))
                print('Red Turn')
            side = -side
            saving(checkers, save)
            WINDOWS_PLOT(WINDOWS, FLIP)
            CHECKER_PLOT(checkers, WINDOWS, FLIP) 
            pg.display.flip()

    # read the last two clicked square and see whether it corresponding any available move
    elif side==1 and len(click)>1:
        if man_jump+king_jump!=[]:
            # man jump
            if tuple(click[-2:]) in man_jump:
                start = click[-2]
                stop = click[-1]
                taken = (start+stop)//2
                checkers[start] = ''
                checkers[taken] = ''
                checkers[stop] = 'm_r'
                man_jump = []
                king_jump = []
                taken_by_man = []
                taken_by_king = []
                MAN_JUMP_RED(stop, checkers, man_jump, taken_by_man)
                print('\tMan jumps from %d to %d.' %(start,stop))
                if man_jump != []:  # red man has a jump sequence
                    continue_to_jump = True
                    print('\tMan continues to jump.')
                else:  # zero or jump sequence ends
                    if stop//Nx == 0:
                        checkers[stop] = 'k_r' #promote
                        print('\tMan promotes.')
                    continue_to_jump = False
                    side = -side
                    print('White Trun')
                    print('==============================================')
                    click = [] # can continue to jump without pressing the jumping piece again (but if the player does 
                              # not clicked on the corrected square that can be jumped, need to repress the jumping piece) 
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
            # king jmup
            elif tuple(click[-2:]) in king_jump:
                start = click[-2]
                stop = click[-1]
                taken = (start+stop)//2
                checkers[start] = ''
                checkers[taken] = ''
                checkers[stop] = 'k_r'
                man_jump = []
                king_jump = []
                taken_by_man = []
                taken_by_king = []
                KING_JUMP_RED(stop, checkers, king_jump, taken_by_king)
                print('\tKing jumps from %d to %d.' %(start,stop))
                if king_jump != []:
                    continue_to_jump = True
                    print('\tKing continues to jump.')
                else:
                    continue_to_jump = False
                    side = -side
                    print('White Trun')
                    print('==============================================')
                    click= [] # can continue to jump without pressing the jumping piece again (but if the player does 
                              # not clicked on the corrected square that can be jumped, need to repress the jumping piece)

                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
        #man walk    
        elif man_walk+king_walk!=[]:
            if tuple(click[-2:]) in man_walk:
                start = click[-2]
                stop = click[-1]  
                checkers[start] = ''
                if stop//Nx == 0:
                    checkers[stop] = 'k_r' #promote
                    print('\tMan walks from %d to %d .' %(start,stop))
                    print('\tMan promotes')
                else:
                    checkers[stop] = 'm_r'
                    print('\tMan walks from %d to %d.' %(start,stop))
                side = -side
                print('White Trun')
                print('==============================================')
                click= []
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
            # king walk
            elif tuple(click[-2:]) in king_walk:
                start = click[-2]
                stop = click[-1]              
                checkers[start] = ''
                checkers[stop] = 'k_r'
                print('\tKing walks from %d to %d.' %(start,stop))
                side = -side
                print('White Turn')
                print('==============================================')
#                saving(checkers, save)
                click= []
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP)
                pg.display.flip()
pg.quit()

NameError: name 'FLIP' is not defined

In [65]:
def EVALUATION(side, list_of_checkers):
    max_eva = 100
    man_r = []
    man_w = []
    king_r = []
    king_w = []
    LIST_CHECKER(list_of_checkers, man_r, man_w, king_r, king_w)
    total_r = len(man_r) + len(king_r)
    total_w = len(man_w) + len(king_w)
    (man_jump, king_jump, man_walk, king_walk, taken_by_man, taken_by_king) = AVAILABLE_MOVE(side, list_of_checkers)
    available_move = man_jump + king_jump + man_walk + king_walk
    if total_r == 0:
        evaluation = max_eva;
    elif total_w == 0 :
        evaluation = -max_eva;
    elif available_move == []:
        if side == 1:
            evaluation = max_eva
        else:
            evaluation = -max_eva
    else:
        evaluation = 1.0*(len(man_w)-len(man_r)) + 1.5*(len(king_w)-len(king_r))
        for i in man_w:
            if i//Nx == Nx-2:
                evaluation += 0.3
            #elif i//Nx == Nx-3:
            #    evaluation += 0.2
        for i in king_w:
            if i//Nx == Nx-1:
                evaluation -= 0.3
            elif i//Nx == Nx-2:
                evaluation -= 0.2
        for i in man_r:
            if i//Nx == 1:
                evaluation -= 0.3
            #elif i//Nx == 2:
            #    evaluation -= 0.2
        for i in king_r:
            if i//Nx == 0:
                evaluation += 0.3
            elif i//Nx == Nx-2:
                evaluation += 0.2
    if side == 1:
        if evaluation!= -max_eva:
            evaluation *= 0.5+total_r/(total_w+total_r)
        evaluation *= -1
    else:
        if evaluation!=max_eva:
            evaluation *= 0.5+total_w/(total_w+total_r)
    return evaluation

def ONE_MOVE(side, list_of_checkers):
    (man_jump, king_jump, man_walk, king_walk, taken_by_man, taken_by_king ) = AVAILABLE_MOVE(side, list_of_checkers)
    checkers_tree = []
    if man_jump+king_jump != []:
        man_jump_tree = []
        man_jump_tree.append((man_jump.copy(),list_of_checkers.copy()))
        while man_jump_tree != []:
            #print(man_jump)
            temp = man_jump_tree[-1][1].copy()
            man_jump = man_jump_tree[-1][0].copy()
            del man_jump_tree[-1]
            for (start, stop) in man_jump:
                virtual_checkers = temp.copy()
                taken = (start+stop)//2
                virtual_checkers[start] = ''
                virtual_checkers[taken] = ''
                if side == 1:
                    if stop//Nx == 0:
                        virtual_checkers[stop] = 'k_r'
                        checkers_tree.append(virtual_checkers.copy())
                    else:
                        virtual_checkers[stop] = 'm_r'              
                        man_jump_prime = []
                        taken_by_man = []
                        MAN_JUMP_RED(stop, virtual_checkers, man_jump_prime, taken_by_man)
                        if man_jump_prime == []:
                            checkers_tree.append(virtual_checkers.copy())
                        else:
                            man_jump_tree.append((man_jump_prime.copy(),virtual_checkers.copy()))
                else:
                    if stop//Nx == Nx-1:
                        virtual_checkers[stop] = 'k_w'
                        checkers_tree.append(virtual_checkers.copy())
                    else:
                        virtual_checkers[stop] = 'm_w'              
                        man_jump_prime = []
                        taken_by_man = []
                        MAN_JUMP_WHITE(stop, virtual_checkers, man_jump_prime, taken_by_man)
                        if man_jump_prime == []:
                            checkers_tree.append(virtual_checkers.copy())
                        else:
                            man_jump_tree.append((man_jump_prime.copy(),virtual_checkers.copy()))
        king_jump_tree = []
        king_jump_tree.append((king_jump.copy(), list_of_checkers.copy()))
        while king_jump_tree != []:
            temp = king_jump_tree[-1][1].copy()
            king_jump = king_jump_tree[-1][0].copy()
            del king_jump_tree[-1]
            for (start, stop) in king_jump:
                virtual_checkers = temp.copy()
                taken = (start+stop)//2
                virtual_checkers[start] = ''
                virtual_checkers[taken] = ''
                if side == 1:
                    virtual_checkers[stop] = 'k_r'              
                    king_jump_prime = []
                    taken_by_king = []
                    KING_JUMP_RED(stop, virtual_checkers, king_jump_prime, taken_by_king)
                    if king_jump_prime == []:
                        checkers_tree.append(virtual_checkers.copy())
                    else:
                        king_jump_tree.append((king_jump_prime.copy(), virtual_checkers.copy()))
                else:
                    virtual_checkers[stop] = 'k_w'              
                    king_jump_prime = []
                    taken_by_king = []
                    KING_JUMP_WHITE(stop, virtual_checkers, king_jump_prime, taken_by_king)
                    if king_jump_prime == []:
                        checkers_tree.append(virtual_checkers.copy())
                    else:
                        king_jump_tree.append((king_jump_prime.copy(), virtual_checkers.copy()))
    elif man_walk+king_walk != []:
        for (start, stop) in man_walk:
            virtual_checkers = list_of_checkers.copy()
            virtual_checkers[start] = ''
            if side == 1:
                if stop//Nx == 0:
                    virtual_checkers[stop] = 'k_r'
                else:
                    virtual_checkers[stop] = 'm_r'
            else:
                if stop//Nx == Nx-1:
                    virtual_checkers[stop] = 'k_w'
                else:
                    virtual_checkers[stop] = 'm_w'
            checkers_tree.append(virtual_checkers.copy())
        for (start, stop) in king_walk:
            virtual_checkers = list_of_checkers.copy()
            virtual_checkers[start] = ''
            if side == 1:
                virtual_checkers[stop] = 'k_r'
            else:
                virtual_checkers[stop] = 'k_w'
            checkers_tree.append(virtual_checkers.copy())
    else:
        checkers_tree.append(list_of_checkers)
    return checkers_tree

In [66]:
def BUILD_TREE(side, list_of_checkers, depth):
    temp = ONE_MOVE(side,list_of_checkers)
    if depth != 1:
        A = []
        for i in temp:
            A.append(BUILD_TREE(-side, i, depth-1))
        return A.copy()
    elif depth == 1:
        point = np.zeros((len(temp)))
        for index,element in enumerate(temp):
            point[index] = EVALUATION(side, element)
        if side == 1:
            return -point
        else:
            return point

def MIN_MAX_SEARCH(side, list_of_checkers, depth):
    temp = ONE_MOVE(side,list_of_checkers)
    point = np.zeros((len(temp)))
    if depth != 1:
        for index, element in enumerate(temp):
            point[index] = MIN_MAX_SEARCH(-side, element, depth-1)
        if side == 1:
            return point.min()
        else:
            return point.max()
    elif depth == 1:
        #print(side)
        for index,element in enumerate(temp):
            point[index] = EVALUATION(side, element)
        if side == 1:
            return -point.max()
        else:
            return point.max()

def FIND_BEST_MOVE_MIN_MAX(side, list_of_checkers, depth):
    temp = ONE_MOVE(side, list_of_checkers)
    point = np.zeros((len(temp)))
    for index,element in enumerate(temp):
        point[index] = MIN_MAX_SEARCH(-side, element, depth-1)
    choice = np.random.choice(np.argwhere(point==point.max()).reshape(-1))
    return (temp[choice].copy(), point.max() )

def ALPHA_BETA_SEARCH(side, list_of_checkers, depth, alpha, beta):
    if depth != 0:
        temp = ONE_MOVE(side, list_of_checkers)
        if side == 1: #minimum node, evaluate beta
            for i in temp:
                if depth != 1:
                    beta = min(beta, ALPHA_BETA_SEARCH(-side, i, depth-1, alpha, beta))
                else:
                    beta = min(beta, ALPHA_BETA_SEARCH(side, i, depth-1, alpha, beta))
                if beta <= alpha:
                    break
            return beta
        else:
            for i in temp:
                if depth != 1:
                    alpha = max(alpha, ALPHA_BETA_SEARCH(-side, i, depth-1, alpha, beta))
                else:
                    alpha = max(alpha, ALPHA_BETA_SEARCH(side, i, depth-1, alpha, beta))
                if beta <= alpha:
                    break
            return alpha
    elif depth == 0:
        point = EVALUATION(side, list_of_checkers)
        if side == 1:
            return -point
        else:
            return point
        
def FIND_BEST_MOVE_ALPHA_BETA(side, list_of_checkers, depth, alpha, beta):
    temp = ONE_MOVE(side, list_of_checkers)
    point = np.zeros((len(temp)))
    for index,element in enumerate(temp):
        point[index] = ALPHA_BETA_SEARCH(-side, element, depth-1, alpha, beta)
    choice = np.random.choice(np.argwhere(point==point.max()).reshape(-1))
    return (temp[choice].copy(), point.max() )

In [67]:
side = -1
depth = 10
#print(MIN_MAX_SEARCH(side, checkers, depth))
#print(ALPHA_BETA_SEARCH(side,checkers, depth, alpha, beta))
print(FIND_BEST_MOVE_ALPHA_BETA(side, checkers, depth, alpha, beta)[1])
print(FIND_BEST_MOVE_MIN_MAX(side, checkers, depth)[1])
#BUILD_TREE(side, checkers, depth)
#FIND_BEST_MOVE_MIN_MAX(1, checkers, depth)

0.0


KeyboardInterrupt: 

### Computer use mini-max and alpha-beta prouning

In [40]:
pg.init()

FLIP = True
FPS = 30 # frame per second
clock = pg.time.Clock()

WINDOWS = pg.display.set_mode((Lx,Ly))
pg.display.set_caption('English Draughts')

#################
checkers = []
for i in range(0,Nx*Nx):
    checkers.append('')
    
#man_r = [10,42,44,46]
#man_w = [12,1,30]
#king_r = [26,28]
#king_w = [39]
checkers = CHECKER_LIST(checkers)  # checkers is original a empty list. The function CHECKER_LIST fills 
                                   # the list with the poistion of red and white pieces delcared (by declaring man_r, man_w, king_r, king_w).
#################
WINDOWS_PLOT(WINDOWS, FLIP)
CHECKER_PLOT(checkers, WINDOWS, FLIP)
pg.display.flip()

click= []
save = []
side = -1
depth = 12
point = 0.0
running = True
continue_to_jump = False


if side==1:
    print('Red Turn')
else:
    print('White Trun')
print('==============================================')
saving(checkers, save)
while running:
    event = pg.event.wait()
    mods = pg.key.get_mods()
    if event.type == MOUSEBUTTONDOWN and event.button==1:
        (x,y) = pg.mouse.get_pos()
        column = (x-margin_x)//grid_size
        row = (y-margin_y)//grid_size
        index = column + row*Nx
        if FLIP:
            index = Nx**2-1-index
        click.append(index)
    # take back
    elif not continue_to_jump and event.type == KEYDOWN and mods&pg.KMOD_CTRL:  # See https://www.pygame.org/docs/ref/key.html
        if event.key == pg.K_z:
            if len(save)>1:
                checkers = save[-2].copy()  # Use copy to clone the element of list. If use '=' directly, what is really cloned
                                            # is the address of the list! See http://www.runoob.com/python3/python3-att-list-copy.html
                del save[-1]  # remove the last element
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP)
                pg.display.flip() 
                print('\tRed takes back.\n==============================================\nRed Turn')
     
    # close the window
    elif event.type == pg.QUIT:
        running = False
        
    if not continue_to_jump:
        (man_jump, king_jump, man_walk, king_walk, taken_by_man, taken_by_king) = AVAILABLE_MOVE(side, checkers)
    available_move = man_jump+king_jump+man_walk+king_walk
    # draw out the hints
    (available_start,available_stop) = MOVE_HINT(available_move)  # find the avaiable start points and stop points, then drawing it out as hint
    if side==1:
        for i in available_start:
            hint_x = i%Nx
            hint_y = i//Nx
            if FLIP:
                hint_x = Nx-1-hint_x
                hint_y = Nx-1-hint_y
            pg.draw.circle(WINDOWS, (0,0,255), 
                           (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
        for i in available_stop:
            hint_x = i%Nx
            hint_y = i//Nx
            if FLIP:
                hint_x = Nx-1-hint_x
                hint_y = Nx-1-hint_y
            pg.draw.circle(WINDOWS, (0,255,0),
                           (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
        pg.display.flip()
    # have a winner
    if available_move == []:
        if side==1:
            white_win = True
            print('No move.')
            print('White win!')
        else:
            red_win = True
            print('No move.')
            print('Red win!')
        break

    # Computer moves
    if side==-1:
        print('Computer is thinking...')
        if point == max_eva and depth>2:
            depth -= 2
        #checkers, point = FIND_BEST_MOVE_MIN_MAX(-1, checkers, depth)  #mini-max
        checkers, point = FIND_BEST_MOVE_ALPHA_BETA(side, checkers, depth, alpha, beta) #alpha-beta
        print('Evaluation = %.2f ; Depth = %d .' %(point,depth) )
        print('Computer finishes thinking.\n==============================================\nRed Turn')
        
        side = -side
        WINDOWS_PLOT(WINDOWS, FLIP)
        CHECKER_PLOT(checkers, WINDOWS, FLIP) 
        pg.display.flip()
        saving(checkers,save)

    # read the last two clicked square and see whether it corresponding any available move
    elif side==1 and len(click)>1:
        if man_jump+king_jump!=[]:
            # man jump
            if tuple(click[-2:]) in man_jump:
                start = click[-2]
                stop = click[-1]
                taken = (start+stop)//2
                checkers[start] = ''
                checkers[taken] = ''
                checkers[stop] = 'm_r'
                man_jump = []
                king_jump = []
                taken_by_man = []
                taken_by_king = []
                MAN_JUMP_RED(stop, checkers, man_jump, taken_by_man)
                print('\tMan jumps from %d to %d.' %(start,stop))
                if man_jump != []:  # red man has a jump sequence
                    continue_to_jump = True
                    print('\tMan continues to jump.')
                else:  # zero or jump sequence ends
                    if stop//Nx == 0:
                        checkers[stop] = 'k_r' #promote
                        print('\tMan promotes.')
                    continue_to_jump = False
                    side = -side
                    print('White Trun')
                    print('==============================================')
                    click = [] # can continue to jump without pressing the jumping piece again (but if the player does 
                              # not clicked on the corrected square that can be jumped, need to repress the jumping piece) 
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
            # king jmup
            elif tuple(click[-2:]) in king_jump:
                start = click[-2]
                stop = click[-1]
                taken = (start+stop)//2
                checkers[start] = ''
                checkers[taken] = ''
                checkers[stop] = 'k_r'
                man_jump = []
                king_jump = []
                taken_by_man = []
                taken_by_king = []
                KING_JUMP_RED(stop, checkers, king_jump, taken_by_king)
                print('\tKing jumps from %d to %d.' %(start,stop))
                if king_jump != []:
                    continue_to_jump = True
                    print('\tKing continues to jump.')
                else:
                    continue_to_jump = False
                    side = -side
                    print('White Trun')
                    print('==============================================')
                    click= [] # can continue to jump without pressing the jumping piece again (but if the player does 
                              # not clicked on the corrected square that can be jumped, need to repress the jumping piece)

                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
        #man walk    
        elif man_walk+king_walk!=[]:
            if tuple(click[-2:]) in man_walk:
                start = click[-2]
                stop = click[-1]  
                checkers[start] = ''
                if stop//Nx == 0:
                    checkers[stop] = 'k_r' #promote
                    print('\tMan walks from %d to %d .' %(start,stop))
                    print('\tMan promotes')
                else:
                    checkers[stop] = 'm_r'
                    print('\tMan walks from %d to %d.' %(start,stop))
                side = -side
                print('White Trun')
                print('==============================================')
                click= []
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
            # king walk
            elif tuple(click[-2:]) in king_walk:
                start = click[-2]
                stop = click[-1]              
                checkers[start] = ''
                checkers[stop] = 'k_r'
                print('\tKing walks from %d to %d.' %(start,stop))
                side = -side
                print('White Turn')
                print('==============================================')
                click= []
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP)
                pg.display.flip()
pg.quit()

White Trun
Computer is thinking...
Evaluation = 0.00 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	Man walks from 44 to 35.
White Trun
Computer is thinking...
Evaluation = 0.00 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	Man walks from 40 to 33.
White Trun
Computer is thinking...
Evaluation = 0.00 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	Man walks from 33 to 26.
White Trun
Computer is thinking...
Evaluation = 0.00 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	Man walks from 26 to 17.
White Trun
Computer is thinking...
Evaluation = 0.00 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	Man jumps from 51 to 37.
	Man continues to jump.
	Man jumps from 37 to 23.
White Trun
Computer is thinking...
Evaluation = 0.00 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	Man walks from 46 to 37.
White Trun
Computer is thinking...
Evaluation = 0.00 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	Man jumps from 53 to 39.
White Trun
Computer is thinking..

	King walks from 17 to 10.
White Turn
Computer is thinking...
Evaluation = 2.08 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	King walks from 28 to 37.
White Turn
Computer is thinking...
Evaluation = 2.08 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	King walks from 10 to 3.
White Turn
Computer is thinking...
Evaluation = 2.08 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	King walks from 37 to 28.
White Turn
Computer is thinking...
Evaluation = 2.08 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	King walks from 28 to 37.
White Turn
Computer is thinking...
Evaluation = 2.08 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	King walks from 37 to 46.
White Turn
Computer is thinking...
Evaluation = 2.08 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	King walks from 3 to 12.
White Turn
Computer is thinking...
Evaluation = 2.08 ; Depth = 12 .
Computer finishes thinking.
Red Turn
	King walks from 12 to 3.
White Turn
Computer is thinking...
Evaluation = 2.08

In [39]:
pg.quit()

### For Debugging

In [43]:
pg.init()

FPS = 30 # frame per second
clock = pg.time.Clock()

WINDOWS = pg.display.set_mode((Lx,Ly))
pg.display.set_caption('English Draughts')

checkers = []
for i in range(0,Nx*Nx):
    checkers.append('')
    

man_r = [24,10,42,44]
man_w = [12,1,30,37]
king_r = [26]
king_w = [62]
side = 1
FLIP = False

checkers = CHECKER_LIST(checkers)

(man_jump, king_jump, man_walk, king_walk, taken_by_man, taken_by_king) = AVAILABLE_MOVE(side, checkers)
#(man_jump, king_jump, man_walk, king_walk, taken_by_man, taken_by_king) = AVAILABLE_MOVE(side, temp)
available_move = man_jump + king_jump + man_walk + king_walk
(available_start,available_stop) = MOVE_HINT(available_move)

WINDOWS_PLOT(WINDOWS, FLIP)
CHECKER_PLOT(checkers, WINDOWS, FLIP)
#CHECKER_PLOT(temp, WINDOWS, flip)
for i in available_start:
    hint_x = i%Nx
    hint_y = i//Nx
    if FLIP:
        hint_x = Nx-1-hint_x
        hint_y = Nx-1-hint_y
    pg.draw.circle(WINDOWS, (0,0,255), (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
for i in available_stop:
    hint_x = i%Nx
    hint_y = i//Nx
    if FLIP:
        hint_x = Nx-1-hint_x
        hint_y = Nx-1-hint_y
    pg.draw.circle(WINDOWS, (0,255,0), (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
pg.display.flip()

running = True
while running:
    clock.tick(FPS)
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False           
    """
    MOUSE TEST
    font = pg.font.Font("./l_10646.ttf", 10)
    if 'k' in checkers[index]:
        text = font.render('it\'s a king', True, (0, 0, 0))
        WINDOWS.blit(text, (x, y))
        pg.display.update()
    elif 'm' in checkers[index]:
        text = font.render('it\'s a man', True, (0, 0, 0))
        WINDOWS.blit(text, (x, y))
        pg.display.update()
    """
pg.quit()

In [82]:
checkers

0.3

In [ ]:
pg.init()

FPS = 10 # frame per second
clock = pg.time.Clock()

WINDOWS = pg.display.set_mode((Lx,Ly))
pg.display.set_caption('English Draughts')

#################
checkers = []
for i in range(0,Nx*Nx):
    checkers.append('')
    
#man_r = [24,10,42,44]
#man_w = [12,1]
#king_r = [26,28]
#king_w = [37]
#################
checkers = CHECKER_LIST(checkers)  # checkers is original a empty list. The function CHECKER_LIST fills 
                                   # the list with the poistion of red and white pieces delcared (by declaring man_r, man_w, king_r, king_w).
#WINDOWS_PLOT(WINDOWS, FLIP)
#CHECKER_PLOT(checkers, WINDOWS, FLIP)
#pg.display.flip()

click= []
save = []
side = 1
running = True
continue_to_jump = False
FLIP = True

if side==-1:
    print('Red Turn')
else:
    print('White Trun')
print('==============================================')
saving(checkers, save)
while running:
    event = pg.event.wait()
    mods = pg.key.get_mods()
    if event.type == MOUSEBUTTONDOWN and event.button==1:
        (x,y) = pg.mouse.get_pos()
        column = (x-margin_x)//grid_size
        row = (y-margin_y)//grid_size
        index = column + row*Nx
        if FLIP:
            index = Nx**2-1-index
        click.append(index)
    # take back
    elif not continue_to_jump and event.type == KEYDOWN and mods&pg.KMOD_CTRL:  # See https://www.pygame.org/docs/ref/key.html
        if event.key == pg.K_z:
            if len(save)>1:
                checkers = save[-2].copy()  # Use copy to clone the element of list. If use '=' directly, what is really cloned
                                            # is the address of the list! See http://www.runoob.com/python3/python3-att-list-copy.html
                del save[-1]  # remove the last element
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP)
                pg.display.flip() 
                print('\tRed takes back.\n==============================================\nRed Turn')
#                else:
#                    print('White takes back.\nWhite Turn')
                
    # close the window
    elif event.type == pg.QUIT:
        running = False
        
    if not continue_to_jump:
        (man_jump, king_jump, man_walk, king_walk, taken_by_man, taken_by_king) = AVAILABLE_MOVE(side, checkers)
    available_move = man_jump+king_jump+man_walk+king_walk
    # draw out the hints
    (available_start,available_stop) = MOVE_HINT(available_move)  # find the avaiable start points and stop points, then drawing it out as hint
    if side==-1:
        for i in available_start:
            hint_x = i%Nx
            hint_y = i//Nx
            if FLIP:
                hint_x = Nx-1-hint_x
                hint_y = Nx-1-hint_y
            pg.draw.circle(WINDOWS, (0,0,255), 
                           (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
        for i in available_stop:
            hint_x = i%Nx
            hint_y = i//Nx
            if FLIP:
                hint_x = Nx-1-hint_x
                hint_y = Nx-1-hint_y
            pg.draw.circle(WINDOWS, (0,255,0),
                           (margin_x+hint_x*grid_size+grid_size//2, margin_y+hint_y*grid_size+grid_size//2), 4, 0 )
        pg.display.flip()
    # have a winner
    if available_move == []:
        if side==1:
            white_win = True
            print('No move.')
            print('White win!')
        else:
            red_win = True
            print('No move.')
            print('Red win!')
        break
    # Computer moves
    if side==-1:
        jump = man_jump + king_jump
        walk = man_walk + king_walk
        if jump!= []:
            selected = np.random.randint(len(jump))
            if selected < len(man_jump):
                start = man_jump[selected][0]
                stop = man_jump[selected][1]
                taken = (start+stop)//2
                checkers[start] = ''
                checkers[taken] = ''
                checkers[stop] = 'm_r'               
                man_jump = []
                #king_jump = []
                taken_by_man = []
                #taken_by_king = []
                MAN_JUMP_WHITE(stop, checkers, man_jump, taken_by_man)
                print('\tMan jumps from %d to %d.' %(start,stop))
                if man_jump != []:  # white man has a jump sequence
                    continue_to_jump = True
                    print('\tMan continues to jump.')
                else:  # zero or jump sequence ends
                    if stop//Nx == Nx-1:
                        checkers[stop] = 'k_r' #promote
                        print('\tMan promotes.')
                    continue_to_jump = False
                    print('Red Turn')
                    print('==============================================')
                    saving(checkers,save)
                    click= []
                    side = -side
            else:
                selected -= len(man_jump)
                start = king_jump[selected][0]
                stop = king_jump[selected][1]
                taken = (start+stop)//2
                checkers[start] = ''
                checkers[taken] = ''
                checkers[stop] = 'k_w'
                #man_jump = []
                king_jump = []
                #taken_by_man = []
                taken_by_king = []
                KING_JUMP_WHITE(stop, checkers, king_jump, taken_by_king)
                print('\tKing jumps from %d to %d.' %(start,stop))
                if king_jump != []:
                    continue_to_jump = True
                    print('\tKing continues to jump.')
                else:
                    continue_to_jump = False
                    print('Red Turn')
                    print('==============================================')
                    saving(checkers, save)
                    click= []
                side = -side        
            WINDOWS_PLOT(WINDOWS, FLIP)
            CHECKER_PLOT(checkers, WINDOWS, FLIP) 
            pg.display.flip()
        elif walk!=[]:
            selected = np.random.randint(len(walk))
            if selected < len(man_walk):
                start = man_walk[selected][0]
                stop = man_walk[selected][1]
                checkers[start] = ''
                if stop//Nx == Nx-1:
                    checkers[stop] = 'k_w' #promote
                    print('\tMan walks from %d to %d .' %(start,stop))
                    print('\tMan promotes')
                else:
                    checkers[stop] = 'm_w'
                    print('\tMan walks from %d to %d.' %(start,stop))
                print('Red Trun')
                print('==============================================')
            else:
                selected -= len(man_walk)
                start = king_walk[selected][0]
                stop = king_walk[selected][1]
                checkers[start] = ''
                checkers[stop] = 'k_w'
                print('\tKing walks from %d to %d.' %(start,stop))
                print('Red Turn')
            side = -side
            saving(checkers, save)
            WINDOWS_PLOT(WINDOWS, FLIP)
            CHECKER_PLOT(checkers, WINDOWS, FLIP) 
            pg.display.flip()

    # read the last two clicked square and see whether it corresponding any available move
    elif side==1 and len(click)>1:
        if man_jump+king_jump!=[]:
            # man jump
            if tuple(click[-2:]) in man_jump:
                start = click[-2]
                stop = click[-1]
                taken = (start+stop)//2
                checkers[start] = ''
                checkers[taken] = ''
                checkers[stop] = 'm_r'
                man_jump = []
                king_jump = []
                taken_by_man = []
                taken_by_king = []
                MAN_JUMP_RED(stop, checkers, man_jump, taken_by_man)
                print('\tMan jumps from %d to %d.' %(start,stop))
                if man_jump != []:  # red man has a jump sequence
                    continue_to_jump = True
                    print('\tMan continues to jump.')
                else:  # zero or jump sequence ends
                    if stop//Nx == 0:
                        checkers[stop] = 'k_r' #promote
                        print('\tMan promotes.')
                    continue_to_jump = False
                    side = -side
                    print('White Trun')
                    print('==============================================')
                    click = [] # can continue to jump without pressing the jumping piece again (but if the player does 
                              # not clicked on the corrected square that can be jumped, need to repress the jumping piece) 
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
            # king jmup
            elif tuple(click[-2:]) in king_jump:
                start = click[-2]
                stop = click[-1]
                taken = (start+stop)//2
                checkers[start] = ''
                checkers[taken] = ''
                checkers[stop] = 'k_r'
                man_jump = []
                king_jump = []
                taken_by_man = []
                taken_by_king = []
                KING_JUMP_RED(stop, checkers, king_jump, taken_by_king)
                print('\tKing jumps from %d to %d.' %(start,stop))
                if king_jump != []:
                    continue_to_jump = True
                    print('\tKing continues to jump.')
                else:
                    continue_to_jump = False
                    side = -side
                    print('White Trun')
                    print('==============================================')
                    click= [] # can continue to jump without pressing the jumping piece again (but if the player does 
                              # not clicked on the corrected square that can be jumped, need to repress the jumping piece)

                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
        #man walk    
        elif man_walk+king_walk!=[]:
            if tuple(click[-2:]) in man_walk:
                start = click[-2]
                stop = click[-1]  
                checkers[start] = ''
                if stop//Nx == 0:
                    checkers[stop] = 'k_r' #promote
                    print('\tMan walks from %d to %d .' %(start,stop))
                    print('\tMan promotes')
                else:
                    checkers[stop] = 'm_r'
                    print('\tMan walks from %d to %d.' %(start,stop))
                side = -side
                print('White Trun')
                print('==============================================')
                click= []
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP) 
                pg.display.flip()
            # king walk
            elif tuple(click[-2:]) in king_walk:
                start = click[-2]
                stop = click[-1]              
                checkers[start] = ''
                checkers[stop] = 'k_r'
                print('\tKing walks from %d to %d.' %(start,stop))
                side = -side
                print('White Turn')
                print('==============================================')
#                saving(checkers, save)
                click= []
                WINDOWS_PLOT(WINDOWS, FLIP)
                CHECKER_PLOT(checkers, WINDOWS, FLIP)
                pg.display.flip()
pg.quit()

White Trun


In [ ]:
pg.init()

FPS = 10 # frame per second
#clock = pg.time.Clock()

#WINDOWS = pg.display.set_mode((Lx,Ly))
#pg.display.set_caption('English Draughts')

#################
checkers = []
for i in range(0,Nx*Nx):
    checkers.append('')
    
#man_r = [24,10,42,44]
#man_w = [12,1]
#king_r = [26,28]
#king_w = [37]
#################
checkers = CHECKER_LIST(checkers)  # checkers is original a empty list. The function CHECKER_LIST fills 
                                   # the list with the poistion of red and white pieces delcared (by declaring man_r, man_w, king_r, king_w).
#WINDOWS_PLOT(WINDOWS, FLIP)
#CHECKER_PLOT(checkers, WINDOWS, FLIP)
#pg.display.flip()

click= []
save = []
side = -1
running = True
continue_to_jump = False
FLIP = True

if side==-1:
    print('Red Turn')
else:
    print('White Trun')
print('==============================================')
saving(checkers, save)


    # Computer moves
if side==-1:
    jump = man_jump + king_jump
    walk = man_walk + king_walk
    if jump!= []:
        selected = np.random.randint(len(jump))
        if selected < len(man_jump):
            start = man_jump[selected][0]
            stop = man_jump[selected][1]
            taken = (start+stop)//2
            checkers[start] = ''
            checkers[taken] = ''
            checkers[stop] = 'm_r'               
            man_jump = []
            #king_jump = []
            taken_by_man = []
            #taken_by_king = []
            MAN_JUMP_WHITE(stop, checkers, man_jump, taken_by_man)
            print('\tMan jumps from %d to %d.' %(start,stop))
            if man_jump != []:  # white man has a jump sequence
                continue_to_jump = True
                print('\tMan continues to jump.')
            else:  # zero or jump sequence ends
                if stop//Nx == Nx-1:
                    checkers[stop] = 'k_r' #promote
                    print('\tMan promotes.')
                continue_to_jump = False
                print('Red Turn')
                print('==============================================')
                saving(checkers,save)
                click= []
''''' side = -side
if side==-1:
        print('Computer is thinking...')
        if point == max_eva and depth>2:
            depth -= 2
        #checkers, point = FIND_BEST_MOVE_MIN_MAX(-1, checkers, depth)  #mini-max
        checkers, point = FIND_BEST_MOVE_ALPHA_BETA(side, checkers, depth, alpha, beta) #alpha-beta
        print('Evaluation = %.2f ; Depth = %d .' %(point,depth) )
        print('Computer finishes thinking.\n==============================================\nRed Turn')
        
        side = -side
        #WINDOWS_PLOT(WINDOWS, FLIP)
        #CHECKER_PLOT(checkers, WINDOWS, FLIP) 
        #pg.display.flip()
        saving(checkers,save) '''''
        
print(checkers)



In [77]:

#FPS = 10 # frame per second
#clock = pg.time.Clock()

#WINDOWS = pg.display.set_mode((Lx,Ly))
#pg.display.set_caption('English Draughts')

#################
checkers = []
checkers1=[]
for i in range(0,Nx*Nx):
    checkers.append('')
    
#man_r = [24,10,42,44]
#man_w = [12,1]
#king_r = [26,28]
#king_w = [37]
#################
checkers = CHECKER_LIST(checkers) 
checkers1 = CHECKER_LIST1(checkers1)# checkers is original a empty list. The function CHECKER_LIST fills 
                                   # the list with the poistion of red and white pieces delcared (by declaring man_r, man_w, king_r, king_w).
#WINDOWS_PLOT(WINDOWS, FLIP)
#CHECKER_PLOT(checkers, WINDOWS, FLIP)
#pg.display.flip()

click= []
save = []
side = 1
running = True
continue_to_jump = False
FLIP = True
click= []
save = []
side = -1
depth = 12
point = 0.0
running = True
continue_to_jump = False


if side==-1:
    print('Red Turn')
else:
    print('White Trun')
print('==============================================')
#saving(checkers, save)


    # Computer moves
if side==1:
    jump = man_jump + king_jump
    walk = man_walk + king_walk
    if jump!= []:
        selected = np.random.randint(len(jump))
        if selected < len(man_jump):
            start = man_jump[selected][0]
            stop = man_jump[selected][1]
            taken = (start+stop)//2
            checkers[start] = ''
            checkers[taken] = ''
            checkers[stop] = 'm_r'               
            man_jump = []
            #king_jump = []
            taken_by_man = []
            #taken_by_king = []
            MAN_JUMP_WHITE(stop, checkers, man_jump, taken_by_man)
            print('\tMan jumps from %d to %d.' %(start,stop))
            if man_jump != []:  # white man has a jump sequence
                continue_to_jump = True
                print('\tMan continues to jump.')
            else:  # zero or jump sequence ends
                if stop//Nx == Nx-1:
                    checkers[stop] = 'k_r' #promote
                    print('\tMan promotes.')
                continue_to_jump = False
                print('Red Turn')
                print('==============================================')
                saving(checkers,save)
                click= []
side = -side


if side==-1:
    print('Red Turn')
else:
    print('White Trun')
print('==============================================')

if side==-1:
        print('Computer is thinking...')
        if point == max_eva and depth>2:
            depth -= 2
        #checkers, point = FIND_BEST_MOVE_MIN_MAX(-1, checkers, depth)  #mini-max
        checkers, point = FIND_BEST_MOVE_ALPHA_BETA(side, checkers, depth, alpha, beta) #alpha-beta
        print('Evaluation = %.2f ; Depth = %d .' %(point,depth) )
        print('Computer finishes thinking.\n==============================================\nRed Turn')
        
        side = -side
        #WINDOWS_PLOT(WINDOWS, FLIP)
        #CHECKER_PLOT(checkers, WINDOWS, FLIP) 
        #pg.display.flip()
        saving(checkers,save) 
        
print(checkers1)




IndexError: list assignment index out of range